In [1]:
using QuantumOptimalControl
using QuantumOptics
using LinearAlgebra
using Flux, DiffEqFlux
using PlotlyJS
using ProgressMeter
using Random
ProgressMeter.ijulia_behavior(:clear)

WebIO._IJuliaInit()

false

In [2]:
bs = SpinBasis(1//2)
sx = sigmax(bs)
ni = 0.5*(identityoperator(bs) + sigmaz(bs));

In [3]:
V = 2π*24.0
δe = -2π*4.5

-28.274333882308138

In [4]:
n_atoms = 12
bsys = tensor([bs for i in 1:n_atoms]...)

H0 = V*sum([embed(bsys, [i, j], [ni, ni])/abs(i-j)^6  for i in 1:n_atoms for j in i+1:n_atoms])
H0 -= δe*sum([embed(bsys, [i], [ni]) for i in [1, n_atoms]])
if n_atoms>8
    H0 -= -2π*1.5*sum([embed(bsys, [i], [ni]) for i in [1, n_atoms]])
    H0 -= -2π*1.5*sum([embed(bsys, [i], [ni]) for i in [4, n_atoms-3]])
end;

H1 = 0.5*sum([embed(bsys, [i], [sx]) for i in 1:n_atoms])
H2 = -sum([embed(bsys, [i], [ni]) for i in 1:n_atoms]);

In [5]:
function GHZ_state(n_atoms)
    state = tensor([spindown(bs)⊗spinup(bs) for i in 1:Int(n_atoms/2)]...) +
            tensor([spinup(bs)⊗spindown(bs) for i in 1:Int(n_atoms/2)]...)
    state/sqrt(2.0)
end 

ground_state(n_atoms) = tensor([spindown(bs) for i in 1:n_atoms]...)
trans = StateTransform(ground_state(n_atoms)=>GHZ_state(n_atoms));

In [6]:
n_neurons = 8
sigmoid(x)= @. 2π*7 / (1 + exp(-x))
Random.seed!(10)
ann = FastChain(FastDense(1, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh), 
                FastDense(n_neurons, 2))
θ = initial_params(ann)  
n_params = length(θ)

106

In [7]:
t0, t1 = 0.0, 0.5

tsf32 = Float32(t0):Float32(t1/49):Float32(t1)
Ωs = Vector{Float32}(2π*vcat(0:0.5:4, 5*ones(32), 4:-0.5:0))
Δs = Vector{Float32}(2π*(-5:10/49:5))
ts = Vector{Float64}(tsf32)

function loss(p)
    c = 0.0f0
    for (i,t) in enumerate(tsf32)
        x = ann([t], p)
        c += (abs(x[1]) - Ωs[i])^2
        c += (x[2] - Δs[i])^2
    end
    #println(c)
    c
end

res = DiffEqFlux.sciml_train(loss, initial_params(ann), ADAM(0.1f0), maxiters = 5000)
θ = Vector{Float64}(res.u);

In [8]:
coeffs(params, t) = let vals = ann([t], params)
                        [abs(vals[1]), vals[2]]
                    end    

cost = CostFunction((x, y) -> 1.0 - abs(sum(conj(x).*y)),
                     p->2e-3*(abs(ann([t0], p)[1])+ 5.0*abs(ann([t1], p)[1])))

CostFunction(var"#23#25"(), var"#24#26"())

In [9]:
H = Hamiltonian(H0, [H1, H2], coeffs);

In [10]:
prob = cu(convert(Float32, QOCProblem(H, trans, (t0, t1), cost)));

In [11]:
@time sol = solve(prob, res.u, ADAM(0.1f0); maxiter=200, abstol=1e-5, reltol=1e-5)


Progress: 100%|█████████████████████████████████████████| Time: 0:23:56
  distance:     0.01712620258331299
  constraints:  0.008312689546361399


1474.028825 seconds (5.51 G allocations: 137.420 GiB, 9.97% gc time, 5.27% compilation time)


Solution{Float32}(Float32[-4.891644, -3.2486708, 3.6208906, -4.285252, -9.749616, -8.146689, 8.857458, -3.5788686, 0.93261456, 1.4900919  …  3.5448282, -6.424723, 7.1616683, -7.219798, 1.7060657, 17.867157, 19.374046, 3.488416, 1.9582546, 6.225233], Float32[0.89193004, 0.8164891, 0.8592043, 0.7698536, 0.79915655, 0.8161896, 0.7245906, 0.6944014, 0.69365954, 0.6437732  …  0.0162189, 0.017019272, 0.020733714, 0.019386351, 0.017857492, 0.01583141, 0.017026603, 0.02026099, 0.018009365, 0.017126203], Float32[0.0110306395, 0.048962075, 0.050294697, 0.03963808, 0.018402087, 0.025846347, 0.017188322, 0.032640316, 0.036211997, 0.02908829  …  0.010170492, 0.005518189, 0.003905754, 0.0028085758, 0.006085555, 0.008987504, 0.005064504, 0.0024155828, 0.0011272868, 0.008312689], Vector{Float32}[])

In [12]:
plot(sol.distance_trace)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [13]:
Ω(t) = abs(ann([t], sol.params)[1])/2π
Δ(t) = ann([t], sol.params)[2]/2π
ts = collect(t0:0.001:t1)

f = plot(
    [
        scatter(x=ts, y=Ω.(ts), name="Ω/2π"),
        scatter(x=ts, y=Δ.(ts), name="Δ/2π"),
    ],
    Layout(
        xaxis_title_text="Time (µs)",
        yaxis_title_text="Frequency (MHz)",
        legend=attr(x=0, y=1,),
        font=attr(
            size=16
        )
    )
)
savefig(f, "GHZ_12_atoms_wfs.eps")

"GHZ_12_atoms_wfs.eps"

In [14]:
tout, psit = schroedinger_dynamic(ts, ground_state(n_atoms), H, Vector{Float64}(sol.params));

In [15]:
f = plot(
    [
        scatter(x=ts, y=real(expect(dm(GHZ_state(n_atoms)), psit))),
    ],
    Layout(
        xaxis_title_text="Time (µs)",
        yaxis_title_text="Overlap (|⟨ψ|GHZ⟩|²)",
        font=attr(
            size=16
        )
    )
)
savefig(f,"GHZ_12_atoms_overlap.eps")

"GHZ_12_atoms_overlap.eps"